# Базовые операции с Kafka

## Подготовка

### Создать топик

In [ ]:
echo "$KAFKA_HOST":"$KAFKA_PORT"

In [ ]:
echo "$KAFKA_STANDBY_HOST":"$KAFKA_STANDBY_PORT"

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic my-mirror-maker-topic \
    --create \
    --replication-factor 1 \
    --partitions 4

### Получить список топиков

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --list

### Записать данные в топик

In [ ]:
kafka-console-producer --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic my-mirror-maker-topic \
    <<<$(yes 'Hello, World!' 2>/dev/null | head -n 10)

### Прочитать данные из топика

In [ ]:
kafka-console-consumer --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic my-mirror-maker-topic \
    --from-beginning \
    --timeout-ms 10000

## Репликация через MirrorMaker

MirrorMaker2 позволяет запустить репликацию между двумя датацентрами:

In [ ]:
SSH_USER=root sshcat /tmp/mirror.maker.properties "${DIND_HOST}" <<EOF
# kafka clusters
clusters = src,tgt
src.bootstrap.servers = kafka-active:9092
tgt.bootstrap.servers = kafka-standby:9092

src->tgt.topics=my-mirror-maker-topic

tgt->src.enabled=false
src->tgt.enabled=true
config.storage.replication.factor=1
status.storage.replication.factor=1
offset.storage.replication.factor=1
checkpoints.topic.replication.factor=1
heartbeats.topic.replication.factor=1
offset-syncs.topic.replication.factor=1
replication.factor=1

replication.policy=org.apache.kafka.connect.mirror.IdentityReplicationPolicy

emit.checkpoints.enables=false
emit.heartbeats.enabled=false
sync.topic.configs.enabled=true
sync.topic.acls.enabled=false
EOF

In [ ]:
docker compose cp /tmp/mirror.maker.properties kafka-active:/tmp/mirror.maker.properties

In [ ]:
connect-mirror-maker /tmp/mirror.maker.properties > mirror-maker.log

Необходимо открыть соседний ноутбук [kafka-consumer.ipynb](kafka-consumer.ipynb) для продолжения работы.

После завершения работы в соседнем ноутбуке необходимо встать на ячейку выше и выбрать в панели меню `Kernel | Interrupt Kernel`, чтобы остановить Mirror Maker